In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_2_Choosing_Investment_Strategies/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 11:59:23.975970


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mean-variance mathematical optimization model for this problem, we'll define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- Let \(N = 10\) be the number of investment strategies.
- Let \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\) be the vector of investment costs.
- Let \(R = [r_{i,j}]\) be the \(N \times M\) matrix of historic returns, where \(r_{i,j}\) is the return of strategy \(i\) in year \(j\), and \(M\) is the number of rows in the "investments_data.csv" file.
- Let \(B = [1000, 1100, 900, 1200]\) be the vector of possible budget scenarios.
- Let \(P = [0.55, 0.4, 0.04, 0.01]\) be the vector of probabilities for each budget scenario.
- Let \(r = 2\) be the risk parameter.

Decision Variables:
- Let \(x_i\) be a binary decision variable indicating whether we invest in strategy \(i\) (\(x_i = 1\)) or not (\(x_i = 0\)).
- Let \(z\) be a continuous decision variable representing the expected return on investment.
- Let \(w\) be a continuous decision variable representing the variance of the return on investment.

Objective Function:
We want to maximize the expected return \(z\) while minimizing the risk \(r \cdot w\). Thus, the objective function is:
\[ \text{maximize } z - r \cdot w \]

Constraints:
1. Budget constraint:
For each budget scenario \(k\), the total investment cost should not exceed the budget:
\[ \sum_{i=1}^{N} A_i \cdot x_i \leq B_k \quad \forall k \in \{1, 2, 3, 4\} \]
2. Probability of exceeding budget constraint:
The probability of exceeding the budget should not exceed 0.4. This can be represented using the Chebyshev's inequality:
\[ \text{Pr} \left( \sum_{i=1}^{N} A_i \cdot x_i > \text{E}[B] \right) \leq \frac{\text{Var}[B]}{(\text{E}[B] - \sum_{i=1}^{N} A_i \cdot x_i)^2} \leq 0.4 \]
where \(\text{E}[B]\) is the expected budget and \(\text{Var}[B]\) is the variance of the budget.
3. Expected return constraint:
The expected return \(z\) should be equal to the weighted average of the historic returns:
\[ z = \frac{1}{M} \sum_{i=1}^{N} \sum_{j=1}^{M} r_{i,j} \cdot x_i \]
4. Variance constraint:
The variance \(w\) should be equal to the weighted average of the historic return variances:
\[ w = \frac{1}{M} \sum_{i=1}^{N} \sum_{j=1}^{M} (r_{i,j} - z)^2 \cdot x_i \]

This formulation provides a mean-variance optimization model that can be used to determine the most optimal investment strategy under the given constraints and uncertain budget scenarios.

In [6]:
print(response_1_text)

To formulate a mean-variance mathematical optimization model for this problem, we'll define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- Let \(N = 10\) be the number of investment strategies.
- Let \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\) be the vector of investment costs.
- Let \(R = [r_{i,j}]\) be the \(N \times M\) matrix of historic returns, where \(r_{i,j}\) is the return of strategy \(i\) in year \(j\), and \(M\) is the number of rows in the "investments_data.csv" file.
- Let \(B = [1000, 1100, 900, 1200]\) be the vector of possible budget scenarios.
- Let \(P = [0.55, 0.4, 0.04, 0.01]\) be the vector of probabilities for each budget scenario.
- Let \(r = 2\) be the risk parameter.

Decision Variables:
- Let \(x_i\) be a binary decision variable indicating whether we invest in strategy \(i\) (\(x_i = 1\)) or not (\(x_i = 0\)).
- Let \(z\) be a continuous decision variable representing the expected return on inve

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here's a Python Pyomo code for the mean-variance optimization problem described:

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]
B = [1000, 1100, 900, 1200]
P = [0.55, 0.4, 0.04, 0.01]
r = 2

# Read historic returns data
investments_data = pd.read_csv("investments_data.csv")
N = len(A)
M = len(investments_data)

R = np.zeros((N, M))
for i in range(N):
    R[i, :] = investments_data.iloc[:, i]

# Define the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(N), domain=pyo.Binary)
model.z = pyo.Var()
model.w = pyo.Var()

# Objective function
model.objective = pyo.Objective(expr=model.z - r * model.w, sense=pyo.maximize)

# Budget constraint
def budget_constraint_rule(model, k):
    return sum(A[i] * model.x[i] for i in range(N)) <= B[k]
model.budget_constraint = pyo.Constraint(range(len(B)), rule=budget_constraint_rule)

# Probability of exceeding budget constraint
E_B = sum(P[k] * B[k] for k in range(len(B)))
Var_B = sum(P[k] * (B[k] - E_B) ** 2 for k in range(len(B)))
model.probability_constraint = pyo.Constraint(expr=Var_B / (E_B - sum(A[i] * model.x[i] for i in range(N))) ** 2 <= 0.4)

# Expected return constraint
model.expected_return_constraint = pyo.Constraint(expr=model.z == (1 / M) * sum(sum(R[i, j] * model.x[i] for j in range(M)) for i in range(N)))

# Variance constraint
model.variance_constraint = pyo.Constraint(expr=model.w == (1 / M) * sum(sum((R[i, j] - model.z) ** 2 * model.x[i] for j in range(M)) for i in range(N)))

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.objective.expr))
print("Decision variables x:", [pyo.value(model.x[i]) for i in range(N)])
print("Expected return z:", pyo.value(model.z))
print("Variance w:", pyo.value(model.w))
```

This code assumes that the "investments_data.csv" file contains the historic returns for each investment strategy. The code defines the Pyomo model with the given parameters, decision variables, objective function, and constraints. It then solves the model using the GLPK solver and prints the results.

In [10]:
print(response_2_text)

Here's a Python Pyomo code for the mean-variance optimization problem described:

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]
B = [1000, 1100, 900, 1200]
P = [0.55, 0.4, 0.04, 0.01]
r = 2

# Read historic returns data
investments_data = pd.read_csv("investments_data.csv")
N = len(A)
M = len(investments_data)

R = np.zeros((N, M))
for i in range(N):
    R[i, :] = investments_data.iloc[:, i]

# Define the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(N), domain=pyo.Binary)
model.z = pyo.Var()
model.w = pyo.Var()

# Objective function
model.objective = pyo.Objective(expr=model.z - r * model.w, sense=pyo.maximize)

# Budget constraint
def budget_constraint_rule(model, k):
    return sum(A[i] * model.x[i] for i in range(N)) <= B[k]
model.budget_constraint = pyo.Constraint(range(len(B)), rule=budget_constraint_rule)

# Probability of exceeding budget con

### Code Executability

In [13]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]
B = [1000, 1100, 900, 1200]
P = [0.55, 0.4, 0.04, 0.01]
r = 2

# Read historic returns data
investments_data = pd.read_csv(FILE_PATH_PROBLEM + "investments_data.csv")
N = len(A)
M = len(investments_data)

R = np.zeros((N, M))
for i in range(N):
    R[i, :] = investments_data.iloc[:, i]

# Define the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(N), domain=pyo.Binary)
model.z = pyo.Var()
model.w = pyo.Var()

# Objective function
model.objective = pyo.Objective(expr=model.z - r * model.w, sense=pyo.maximize)

# Budget constraint
def budget_constraint_rule(model, k):
    return sum(A[i] * model.x[i] for i in range(N)) <= B[k]
model.budget_constraint = pyo.Constraint(range(len(B)), rule=budget_constraint_rule)

# Probability of exceeding budget constraint
E_B = sum(P[k] * B[k] for k in range(len(B)))
Var_B = sum(P[k] * (B[k] - E_B) ** 2 for k in range(len(B)))
model.probability_constraint = pyo.Constraint(expr=Var_B / (E_B - sum(A[i] * model.x[i] for i in range(N))) ** 2 <= 0.4)

# Expected return constraint
model.expected_return_constraint = pyo.Constraint(expr=model.z == (1 / M) * sum(sum(R[i, j] * model.x[i] for j in range(M)) for i in range(N)))

# Variance constraint
model.variance_constraint = pyo.Constraint(expr=model.w == (1 / M) * sum(sum((R[i, j] - model.z) ** 2 * model.x[i] for j in range(M)) for i in range(N)))

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.objective.expr))
print("Decision variables x:", [pyo.value(model.x[i]) for i in range(N)])
print("Expected return z:", pyo.value(model.z))
print("Variance w:", pyo.value(model.w))

ValueError: Cannot write legal LP file.  Constraint 'probability_constraint' has a body with nonlinear terms.

### Solution Correctness